In [ ]:
import pandas as pd

### A partir d'un dataFrame cette methode va récuperer toute les cartes joués et en faire des phrases qui seront en suite passé à l'algorithme word2vec

### A games 
### On prend la colonne games

### A partir de card_history
### on recupere l'historique des cartes sorties pour chaque partie On filtre si le champs card_history existe ou si il est vide

### A partir de sentances
### Pour chaque partie on rassemble toutes les cartes sorties par "me" et "opponent", puis on recupère le nom des cartes pour crée des phrases.


In [ ]:
def createsentances(data):
    games = data["games"]
    cards = [e["card_history"] for e in games if e.has_key("card_history") and e["card_history"]!=[]]
    sentances = []
    for c in cards:
        sentances.append([e["card"]["name"]for e in c if e["player"] == "me"])
        sentances.append([e["card"]["name"]for e in c if e["player"] == "opponent"])
    return sentances

creation de phrase a partir de plusieurs fichier json
listfile est une liste de fichier avec leur chemin

In [ ]:
def createsentancesfiles(listfile):
    sentances = []
    for l in listfile:
        data=pd.read_json(l)
        sentances += createsentances(data)
        del data
    return sentances

In [ ]:
directoryHearthstoneGames = "HeathstoneGames/"
from os import listdir
from os.path import isfile, join
filesHearthstoneGames = [directoryHearthstoneGames+f for f in listdir(directoryHearthstoneGames) if isfile(join(directoryHearthstoneGames, f))]


In [ ]:
sentances = createsentancesfiles(filesHearthstoneGames)

In [ ]:
data = pd.read_json("2018-01.zip");

In [ ]:
sentances = createsentances(data)


# Je prend l'url avec collectible car je ne veux que les cartes disponibles pour faire un deck

In [ ]:
import requests

urlApi = "https://api.hearthstonejson.com/v1/latest/enUS/cards.collectible.json"

ApiHearthStone = requests.get(urlApi).json()

In [ ]:
listNameCard = [e["name"] for e in ApiHearthStone]

on filtre les cartes des parties à seulement celle disponible pour la construction de deck

In [ ]:
sentances = [filter(lambda x: x in listNameCard, e) for e in sentances]

# On va utilisier gensim et plus particulièrement l'algo Word2vec

In [ ]:
from gensim.models import Word2Vec

max_count exclue les mots dont leur nombre d'apparition est inférieure à celui indiqué
size nombre de noeuds
workers paralléliser sur un nombres de coeurs
ier nombre d'epochs

In [ ]:
model = Word2Vec(sentances,min_count=10,size=200,workers=4,iter=100)

sauvegarde du model

In [ ]:
model.save("word2vecModel")

charger le modele entrainer et de continuer l'entrainement

In [ ]:
model = Word2Vec.load("word2vecModel")

on filtre l'api car nous avons pas besoin des information sur les héros qui ne sont pas joué pas joués par les joueurs
on cree un dictionnaire pour faire correspondre le nom d'une carte à sa classe
utile apres pour verifier que les cartes suggere sont du meme classe que celle déjà dans le deck

In [ ]:
cardKnowns = model.wv.vocab.keys()
ApiHearthStone = filter(lambda x:x["name"] in cardKnowns,ApiHearthStone)
cardClass = {x["name"]:x["cardClass"] for x in ApiHearthStone}

fonction qui creer un deck aleatoire
numberscards nombre de carte voulue
typecard le type de deck
cardClass dictionnaire de carte associer à leur classe

In [ ]:
import random
def createDeck(cardClass,typecard,numbercards):
    deck = []
    n = [k for k,v in cardClass.iteritems() if(v==typecard or v == "NEUTRAL")]
    
    for i in range(numbercards):
        indice = random.randint(0,len(n)-1)
        deck.append(n[indice])
    return deck

proposition de cartes, si deck de taille 30 ou plus aucune proposition

In [ ]:
def proposeCards(uncompletedDeck,model,cardsClass,nbproposes=2):
    deckClass = ""
    uncompletedDeckInside = list(uncompletedDeck)
    numberMaxCards = 30
    if len(uncompletedDeckInside) <1:
        raise ValueError("it need to have at least 1 card in the uncompleted deck")
    if len(uncompletedDeckInside) >= numberMaxCards:
        return
    if nbproposes < 1:
        raise ValueError("nbcards can't be under 1")
    if nbproposes>5:
        nbproposes = 5
    
    #on verifie qu des le départ que 2 classes de cartes ne soient pas presente
    #exception à la classe NEUTRAL
    
    nbClasses = set([cardsClass[e] for e in myuncompletedDeck])
    if len(nbClasses)>2 or (len(nbClasses)==2 and "NEUTRAL" not in nbClasses):
       raise ValueError("Just one class can be present in the uncompleted deck.\n Exeception for neutrals cards")
    
    #on met a jour cardClass qui correpond à la classe du deck en fonction de la classe de cartes proposé 
    #si toutes les cartes sont neutre
    deckClass = [e for e in nbClasses if e != "NEUTRAL"]
    #si NEUTRAL
    if deckClass==[]:
        deckClass = ""
    #si classe
    else:
        deckClass = deckClass[0]
    
    #on va essayer de definir une classe de deck en fonction de la proposition on prend les premières jusqu'a une
    #carte non neutre
    
    negativeCards = []
    
    while len(uncompletedDeckInside)<numberMaxCards:
        proposes = model.wv.most_similar(positive=uncompletedDeckInside,negative=negativeCards,topn=nbproposes)
        proposes = [x[0] for x in proposes]
        
        if(deckClass==""):
            for e in proposes:
                if cardsClass[e] != "NEUTRAL":
                    deckClass = cardsClass[e]
                    break
        
        negativeCards += [e for e in proposes if cardsClass[e] != "NEUTRAL" and cardsClass[e]!=deckClass]
        proposes = [e for e in proposes if cardsClass[e] == "NEUTRAL" or cardsClass[e]==deckClass]
        
        diff = numberMaxCards-len(uncompletedDeckInside)
        if diff < nbproposes :
            proposes = proposes[:diff]
       
        uncompletedDeckInside += proposes 
    return uncompletedDeckInside

In [ ]:
myuncompletedDeck = ["The Lich King","Loatheb","Abomination"]

decks= []

for i in range(1,6):
    decks.append(proposeCards(myuncompletedDeck,model,cardClass,nbproposes=i))

[len(e) for e in decks]



In [ ]:
myuncompletedDeck = createDeck(cardClass,"PALADIN",16)

In [ ]:
decks= []

for i in range(1,12):
    decks.append(proposeCards(myuncompletedDeck,model,cardClass,nbproposes=i))

[len(e) for e in decks]

In [ ]:
set([cardClass[e] for e in decks[0]])

In [ ]:
for d in decks:
    print set([cardClass[e] for e in d])